In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/mouse_lymph3_ctrl_visiumHD_subsampled.h5ad')
adata

AnnData object with n_obs × n_vars = 10000 × 2707
    obs: 'cell_type', 'leiden', 'cell_type_int', 'sample'
    uns: 'cell_thresholds', 'received_ligands', 'received_ligands_tfl'
    obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_unscaled', 'tangram_ct_pred'
    varm: 'PCs'
    layers: 'imputed_count', 'normalized_count', 'raw_count'

In [4]:
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append('../../src')

In [5]:
from spaceoracle.tools.network import RegulatoryFactory
co_grn = RegulatoryFactory(
    colinks_path='/ix/djishnu/shared/djishnu_kor11/training_data_2025/mouse_lymph3_ctrl_visiumHD_subsampled_colinks.pkl',
    annot='cell_type_int'
)

In [6]:
from spaceoracle.models.parallel_estimators import SpatialCellularProgramsEstimator

estimator = SpatialCellularProgramsEstimator(
    adata, 
    target_gene='Cd74', 
    cluster_annot='cell_type_int',
    radius=400, 
    contact_distance=50,
    grn=co_grn,
)

In [7]:
estimator.lr

,ligand,receptor,pathway,signaling,radius,pairs
0,Tgfb1,Tgfbr1,TGFb,Secreted Signaling,400,Tgfb1$Tgfbr1
0,Tgfb1,Tgfbr2,TGFb,Secreted Signaling,400,Tgfb1$Tgfbr2
2,Tgfb3,Tgfbr1,TGFb,Secreted Signaling,400,Tgfb3$Tgfbr1
2,Tgfb3,Tgfbr2,TGFb,Secreted Signaling,400,Tgfb3$Tgfbr2
3,Tgfb1,Acvr1b,TGFb,Secreted Signaling,400,Tgfb1$Acvr1b
...,...,...,...,...,...,...
2012,Thy1,Itgb3,THY1,Cell-Cell Contact,50,Thy1$Itgb3
2015,Itga4,Vcam1,VCAM,Cell-Cell Contact,50,Itga4$Vcam1
2015,Itgb1,Vcam1,VCAM,Cell-Cell Contact,50,Itgb1$Vcam1
2016,Itga9,Vcam1,VCAM,Cell-Cell Contact,50,Itga9$Vcam1


In [8]:
estimator.fit(num_epochs=100, learning_rate=5e-3, 
    score_threshold=0.2, l1_reg=1e-9, estimator='lasso')

Fitting Cd74 with 1036 modulators
	94 Transcription Factors
	724 Ligand-Receptor Pairs
	218 TranscriptionFactor-Ligand Pairs


NameError: name 'use_anchors' is not defined

In [ ]:
base_dir = '/ix/djishnu/shared/djishnu_kor11/'
from spaceoracle import SpaceTravLR

star = SpaceTravLR(
    adata=adata,
    annot='cell_type_int', 
    max_epochs=100, 
    learning_rate=5e-3, 
    spatial_dim=64,
    batch_size=512,
    grn=co_grn,
    radius=800,
    contact_distance=50,
    save_dir=base_dir + 'lasso_runs/mouse_lymph3_visiumHD_subsampled'
)

star.run()